Replication of this [kaggle notebook](https://www.kaggle.com/janniskueck/pm3-notebook-inference-clustering)

In [278]:
# Import relevant packages
import pandas as pd
import numpy as np
import pyreadr
from sklearn import preprocessing
import patsy

from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

import hdmpy
import numpy as np
import random
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors

In [279]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import itertools
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_categorical_dtype
from itertools import compress
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.feature_selection import SelectFromModel

This notebook contains an example for teaching.

# A Case Study: The Effect of Gun Ownership on Gun-Homicide Rates

We consider the problem of estimating the effect of gun
ownership on the homicide rate. For this purpose, we estimate the following partially
linear model

$$
 Y_{j,t} = \beta D_{j,(t-1)} + g(Z_{j,t}) + \epsilon_{j,t}.
$$

## Data

$Y_{j,t}$ is log homicide rate in county $j$ at time $t$, $D_{j, t-1}$ is log  fraction of suicides committed with a firearm in county $j$ at time $t-1$, which we use as a proxy for gun ownership,  and  $Z_{j,t}$ is a set of demographic and economic characteristics of county $j$ at time $t$. The parameter $\beta$ is the effect of gun ownership on the
homicide rates, controlling for county-level demographic and economic characteristics. 

The sample covers 195 large United States counties between the years 1980 through 1999, giving us 3900 observations.

In [295]:
data1 = pd.read_csv( r"../data/gun_clean.csv" )
data1.shape[1]

415

### Preprocessing

To account for heterogeneity across counties and time trends in  all variables, we remove from them county-specific and time-specific effects in the following preprocessing.

In [296]:
import re

In [299]:
#################################  Find Variable Names from Dataset ########################

def varlist( df = None, type_dataframe = [ 'numeric' , 'categorical' , 'string' ],  pattern = "", exclude = None ):
    varrs = []
    
    if ('numeric' in type_dataframe):
        varrs = varrs + df.columns[ df.apply( is_numeric_dtype , axis = 0 ).to_list() ].tolist()
    
    if ('categorical' in type_dataframe):
        varrs = varrs + df.columns[ df.apply( is_categorical_dtype , axis = 0 ).to_list() ].tolist()
    
    if ('string' in type_dataframe):
        varrs = varrs + df.columns[ df.apply( is_string_dtype , axis = 0 ).to_list() ].tolist()
    
    grepl_result = np.array( [ re.search( pattern , variable ) is not None for variable in df.columns.to_list() ] )
    
    if exclude is None:
        result = list(compress( varrs, grepl_result ) )
    
    else:
        varrs_excluded = np.array( [var in exclude for var in varrs ] )
        and_filter = np.logical_and( ~varrs_excluded ,  grepl_result )
        result = list(compress( varrs, and_filter ) )
    
    return result   

################################# Create Variables ###############################


# Dummy Variables for Year and County Fixed Effects
r = re.compile("X_Jfips")
fixed = list( filter( r.match, data1.columns.to_list() ) )
year = varlist(data1, pattern="X_Tyear")

census = []
census_var = ["^AGE", "^BN", "^BP", "^BZ", "^ED", "^EL","^HI", "^HS", "^INC", "^LF", "^LN", "^PI", "^PO", "^PP", "^PV", "^SPR", "^VS"]

for variable in census_var:
    r = re.compile( variable )
    census = census + list( filter( r.match, data1.columns.to_list() ) )

    
################################ Variables ##################################
# Treatment Variable
d = "logfssl"

# Outcome Variable
y = "logghomr"

# Other Control Variables
X1 = ["logrobr", "logburg", "burg_missing", "robrate_missing"]
X2 = ["newblack", "newfhh", "newmove", "newdens", "newmal"]

#################################  Partial out Fixed Effects ########################

rdata = data1[['CountyCode']]

# Variables to be Partialled-out
varlist2 = [y] + [d] + X1 + X2 + census

# Partial out Variables in varlist from year and county fixed effect
for var_name in varlist2:
    form = var_name + " ~ "  + " + ".join( year ) + "+" + " + ".join( fixed )
    rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid

<ipython-input-299-165111bd83a4>:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid


In [298]:
# load dataset
rdata_read = pyreadr.read_r("../data/gun_clean.RData")
data = rdata_read[ 'data' ]
n = data.shape[0]

## We check that our results are equal to R results at 6 decimals

In [300]:
column_names = data.columns.to_list()

for col in column_names:
    result = (data[f'{col}'].round(6) == rdata[f'{col}'].round(6)).sum()
    
    if result == 3900:
        print(f'Column {col} are equal at 6 decimals.')
    else:
        print(f'\n\nColumn {col} are not equal at 6 decimals.')

Column logghomr are equal at 6 decimals.
Column logfssl are equal at 6 decimals.
Column logrobr are equal at 6 decimals.
Column logburg are equal at 6 decimals.
Column burg_missing are equal at 6 decimals.
Column robrate_missing are equal at 6 decimals.
Column newblack are equal at 6 decimals.
Column newfhh are equal at 6 decimals.
Column newmove are equal at 6 decimals.
Column newdens are equal at 6 decimals.
Column newmal are equal at 6 decimals.
Column AGE010D are equal at 6 decimals.
Column AGE050D are equal at 6 decimals.
Column AGE110D are equal at 6 decimals.
Column AGE170D are equal at 6 decimals.
Column AGE180D are equal at 6 decimals.
Column AGE270D are equal at 6 decimals.
Column AGE310D are equal at 6 decimals.
Column AGE320D are equal at 6 decimals.
Column AGE350D are equal at 6 decimals.
Column AGE380D are equal at 6 decimals.
Column AGE410D are equal at 6 decimals.
Column AGE470D are equal at 6 decimals.
Column AGE570D are equal at 6 decimals.
Column AGE640D are equal at

Now, we can construct the treatment variable, the outcome variable and the matrix $Z$ that includes the control variables.

In [301]:
# Treatment Variable
D = rdata[ f'{d}']

# Outcome Variable
Y = rdata[ f'{y}']

# Construct matrix Z
Z = rdata[ X1 + X2 + census ]

Z.shape

(3900, 195)

We have in total 195 control variables. The control variables $Z_{j,t}$ are from the U.S. Census Bureau and  contain demographic and economic characteristics of the counties such as  the age distribution, the income distribution, crime rates, federal spending, home ownership rates, house prices, educational attainment, voting paterns, employment statistics, and migration rates. 

In [302]:
clu = rdata[['CountyCode']]
data = pd.concat([Y, D, Z, clu], axis=1)

In [303]:
data.to_csv( r"../data/gun_clean2.csv" , index = False )

## The effect of gun ownership

### OLS

After preprocessing the data, we first look at simple regression of $Y_{j,t}$ on $D_{j,t-1}$ without controls as a baseline model.

In [304]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

In [305]:
# # Run this line to avoid all the lines of code above
# data = pd.read_csv( r"../data/gun_clean2.csv"  )

In [306]:
# OLS clustering at the County level
model = "logghomr ~ logfssl"
baseline_ols = smf.ols(model , data=data).fit().get_robustcov_results(cov_type = "cluster", groups= data['CountyCode'])
baseline_ols_table = baseline_ols.summary2().tables[1]
print( baseline_ols_table.iloc[ 1 , 4:] )
baseline_ols_table.iloc[1, :]

[0.025    0.154480
0.975]    0.410129
Name: logfssl, dtype: float64


Coef.       0.282304
Std.Err.    0.064811
t           4.355825
P>|t|       0.000021
[0.025      0.154480
0.975]      0.410129
Name: logfssl, dtype: float64

The point estimate is $0.282$ with the confidence interval ranging from 0.155 to 0.41. This
suggests that increases in gun ownership rates are related to gun homicide rates - if gun ownership increases by 1% relative
to a trend then the predicted gun homicide rate goes up by 0.28%, without controlling for counties' characteristics.

Since our goal is to estimate the effect of gun ownership after controlling for a rich set county characteristics we next include the controls. First, we estimate the model by ols and then by an array of the modern regression methods using the double machine learning approach.

In [307]:
control_formula = "logghomr" + ' ~ ' + 'logfssl + ' + ' + '.join( Z.columns.to_list() )
control_formula

'logghomr ~ logfssl + logrobr + logburg + burg_missing + robrate_missing + newblack + newfhh + newmove + newdens + newmal + AGE010D + AGE050D + AGE110D + AGE170D + AGE180D + AGE270D + AGE310D + AGE320D + AGE350D + AGE380D + AGE410D + AGE470D + AGE570D + AGE640D + AGE670D + AGE760D + BNK010D + BNK050D + BPS030D + BPS130D + BPS230D + BPS020D + BPS120D + BPS220D + BPS820D + BZA010D + BZA110D + BZA210D + EDU100D + EDU200D + EDU600D + EDU610D + EDU620D + EDU630D + EDU635D + EDU640D + EDU650D + EDU680D + EDU685D + ELE010D + ELE020D + ELE025D + ELE030D + ELE035D + ELE060D + ELE065D + ELE210D + ELE220D + HIS010D + HIS020D + HIS030D + HIS040D + HIS110D + HIS120D + HIS130D + HIS140D + HIS200D + HIS300D + HIS500D + HIS700D + HSD010D + HSD020D + HSD030D + HSD110D + HSD120D + HSD130D + HSD140D + HSD150D + HSD170D + HSD200D + HSD210D + HSD230D + HSD300D + HSD310D + HSG030D + HSG195D + HSG200D + HSG220D + HSG440D + HSG445D + HSG460D + HSG680D + HSG700D + HSD410D + HSD500D + HSD510D + HSD520D + HSD530

In [ ]:
control_ols = smf.ols( control_formula , data=data).fit().get_robustcov_results(cov_type = "cluster", groups= data['CountyCode'])
control_ols_table = control_ols.summary2().tables[1]
print( control_ols_table.iloc[ 1 , 4:] )
control_ols_table.iloc[1, :]

After controlling for a rich set of characteristics, the point estimate of gun ownership reduces to $0.19$.

# DML algorithm

Here we perform inference of the predictive coefficient $\beta$ in our partially linear statistical model, 

$$
Y = D\beta + g(Z) + \epsilon, \quad E (\epsilon | D, Z) = 0,
$$

using the **double machine learning** approach. 

For $\tilde Y = Y- E(Y|Z)$ and $\tilde D= D- E(D|Z)$, we can write
$$
\tilde Y = \alpha \tilde D + \epsilon, \quad E (\epsilon |\tilde D) =0.
$$

Using cross-fitting, we employ modern regression methods
to build estimators $\hat \ell(Z)$ and $\hat m(Z)$ of $\ell(Z):=E(Y|Z)$ and $m(Z):=E(D|Z)$ to obtain the estimates of the residualized quantities:

$$
\tilde Y_i = Y_i  - \hat \ell (Z_i),   \quad \tilde D_i = D_i - \hat m(Z_i), \quad \text{ for each } i = 1,\dots,n.
$$

Finally, using ordinary least squares of $\tilde Y_i$ on $\tilde D_i$, we obtain the 
estimate of $\beta$.

The following algorithm comsumes $Y, D, Z$, and a machine learning method for learning the residuals $\tilde Y$ and $\tilde D$, where the residuals are obtained by cross-validation (cross-fitting). Then, it prints the estimated coefficient $\beta$ and the corresponding standard error from the final OLS regression.

In [308]:
# as matrix
y = Y.to_numpy().reshape( len(Y) , 1 )
d = D.to_numpy().reshape( len(Y) , 1 )
z = Z.to_numpy()
clu = clu.to_numpy().reshape( len(Y) , 1 )

In [309]:
def DML2_for_PLM(z, d, y, dreg, yreg, nfold, clu):
    
    # Num ob observations
    nobs = z.shape[0]
    
    # Define folds indices 
    list_1 = [*range(0, nfold, 1)]*nobs
    sample = np.random.choice(nobs,nobs, replace=False).tolist()
    foldid = [list_1[index] for index in sample]

    # Create split function(similar to R)
    def split(x, f):
        count = max(f) + 1
        return tuple( list(itertools.compress(x, (el == i for el in f))) for i in range(count) ) 

    # Split observation indices into folds 
    list_2 = [*range(0, nobs, 1)]
    I = split(list_2, foldid)
    
    # Create array to save errors 
    dtil = np.zeros( len(z) ).reshape( len(z) , 1 )
    ytil = np.zeros( len(z) ).reshape( len(z) , 1 )
    
    # loop to save results
    for b in range(0,len(I)):
    
        # Split data - index to keep are in mask as booleans
        include_idx = set(I[b])  #Here should go I[b] Set is more efficient, but doesn't reorder your elements if that is desireable
        mask = np.array([(i in include_idx) for i in range(len(z))])

        # Lasso regression, excluding folds selected 
        dfit = dreg(z[~mask,], d[~mask,])
        yfit = yreg(z[~mask,], y[~mask,])

        # predict estimates using the 
        dhat = dfit.predict( z[mask,] )
        yhat = yfit.predict( z[mask,] )

        # save errors  
        dtil[mask] =  d[mask,] - dhat.reshape( len(I[b]) , 1 )
        ytil[mask] = y[mask,] - yhat.reshape( len(I[b]) , 1 )
        print(b, " ")
    
    # Create dataframe 
    data_2 = pd.DataFrame(np.concatenate( ( ytil, dtil,clu ), axis = 1), columns = ['ytil','dtil','CountyCode'])
   
    # OLS clustering at the County level
    model = "ytil ~ dtil"
    baseline_ols = smf.ols(model , data=data).fit().get_robustcov_results(cov_type = "cluster", groups= data_2['CountyCode'])
    coef_est = baseline_ols.summary2().tables[1]['Coef.']['dtil']
    se = baseline_ols.summary2().tables[1]['Std.Err.']['dtil']
    
    Final_result = { 'coef_est' : coef_est , 'se' : se , 'dtil' : dtil , 'ytil' : ytil }

    print("Coefficient is {}, SE is equal to {}".format(coef_est, se))
    
    return Final_result
    

# Lasso 

In [315]:
def dreg(z,d):
    alpha=0.00000001
    result = linear_model.Lasso(alpha = alpha).fit(z, d)
    return result

def yreg(z,y):
    alpha=0.00000001
    result = linear_model.Lasso(alpha = alpha).fit(z, y)
    return result

DML2_lasso = DML2_for_PLM(z, d, y, dreg, yreg, 10, clu)

c:\python\python38\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11.674735395967629, tolerance: 0.002864909854321656
  model = cd_fast.enet_coordinate_descent(
c:\python\python38\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 140.81523453499932, tolerance: 0.03618879801918865
  model = cd_fast.enet_coordinate_descent(


0  


c:\python\python38\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.682128784205268, tolerance: 0.0025091298640074386
  model = cd_fast.enet_coordinate_descent(


1  
Coefficient is 0.22828848301587235, SE is equal to 0.07867826261327843


c:\python\python38\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 130.44928205824675, tolerance: 0.03296600864704124
  model = cd_fast.enet_coordinate_descent(


#### We use SelectFromModel to select variables which do not have a zero coefficient. It is done because we want to reduce dimensionality as rlasso( post = T ) does.

In [316]:
class Lasso_post:
    
    def __init__(self, alpha ):
        self.alpha = alpha

        
    def fit( self, X, Y ):
        self.X = X
        self.Y = Y
        lasso = linear_model.Lasso( alpha = alpha ).fit( X , Y )
        model = SelectFromModel( lasso , prefit = True )
        X_new = model.transform( X )
        # Gettin indices from columns which has variance for regression
        index_X = model.get_support()
        
        self.index = index_X
        new_x = X[ : ,  index_X ]
        
        lasso2 = linear_model.Lasso( alpha = alpha ).fit( new_x , Y )
        self.model = lasso2
        
        return self
    
    def predict( self , X ):
        
        dropped_X = X[ : , self.index ]
        
        predictions = self.model.predict( dropped_X )
        
        return predictions

Run the below code to verify whether Lasso_post functions as it is expected. 

In [181]:
# # setting alpha 
# alpha = 0.00000001

# # fit original model
# lasso = linear_model.Lasso( alpha = alpha ).fit( z , d )
# # select non zero coefficient variables
# model = SelectFromModel( lasso , prefit = True )
# # Dropping zero coefficient variables
# z_new = model.transform( z )
# # Getting the index of non zero coefficient variables
# index_X = model.get_support()
# # fitting a new model with the selected variables
# lasso2 = linear_model.Lasso( alpha = alpha ).fit( z[ : ,  index_X ] , d )
# # predictions with selected variables
# resultado = lasso2.predict( z[ : ,  index_X ] )

# # Making rlasso( post = T ) with Lasso_post 
# model1 = Lasso_post( alpha = alpha )
# mdfit = model1.fit( z, d )
# resultado2 = mdfit.predict( z )

# np.sum(resultado == resultado2)
# # the number of values which are equal are equal to the number of observations.
# # We conclude that our new class Lasso_post runs what we desire.

In [ ]:
def dreg(z,d):
    alpha=0.00000001
    result = Lasso_post( alpha = alpha ).fit( z , d )
    return result

def yreg( z , y ):
    alpha = 0.00000001
    result = Lasso_post( alpha = alpha ).fit( z , y )
    return result

DML2_lasso_post = DML2_for_PLM(z, d, y, dreg, yreg, 10, clu)

### cv.glmnet curiosities
1. According to [link1](https://stackoverflow.com/questions/24098212/what-does-the-option-normalize-true-in-lasso-sklearn-do) and [link2](https://statisticaloddsandends.wordpress.com/2018/11/15/a-deep-dive-into-glmnet-standardize/), It standardize **X** variables before estimation. In sklearn we have an option named **normalize**. It normalizes **X** by subtracting the mean and dividing by the l2-norm. Based on [link3](https://stackoverflow.com/questions/59846325/confusion-about-standardize-option-of-glmnet-package-in-r), **cv.glmnet (standardize = TRUE)** and **sklearn (normlize = True)** are not the same. We decided to use **StandardScaler** that meets suggestions made in **link3**. We proved this in the commented code below.

In [ ]:
# a = np.array([ 6, 7, 8, 12, 15]).reshape( a.size, 1)

# scaler_a = StandardScaler()
# scaler_a.fit( a )
# print( scaler_a.transform( a ) )

# def std_1( x ):
#     val = ( (1 / x.size )*np.sum( ( x - np.mean( x ) ) ** 2 ) ) ** 0.5
#     return val

# ( a - np.mean(a) ) / std_1( a )

In [318]:
class standard_skl_model:
    
    def __init__(self, model ):
        self.model = model
       
    def fit( self, X, Y ):
        
        # Standarization of X and Y
        self.scaler_X = StandardScaler()
        self.scaler_X.fit( X )
        std_X = self.scaler_X.transform( X )
                
        self.model.fit( std_X , Y )
                
        return self
    
    def predict( self , X ):
        
        self.scaler_X = StandardScaler()
        self.scaler_X.fit( X )
        std_X = self.scaler_X.transform( X )
        
        prediction = self.model.predict( std_X )
        
        return prediction

Run the below code to verify whether standard_skl_model functions as it is expected. 

In [319]:
# # Regular Operation using StandardScaler
# # Standarization of z
# scaler_z = StandardScaler()
# scaler_z.fit( z )
# std_z = scaler_z.transform( z )
# # Lasso CV prediction and getting normal 
# model1 = LassoCV(cv = 10 , random_state = 0 ).fit( std_z, std_d )
# predict1 = model1.predict( std_z )

# # Class standard_skl_model
# model2 = standard_skl_model( LassoCV(cv = 10 , random_state = 0 ) ).fit( z, d)
# predict2 = model2.predict( z )

# # Checking two results
# np.sum(real_predict1 == real_predict2)
# # the number of values which are equal are equal to the number of observations.
# # We conclude that our new class standard_skl_model runs what we desire.

In [ ]:
#DML with cross-validated Lasso:
def dreg(z,d):
    result = standard_skl_model( LassoCV(cv = 10 , random_state = 0 ) ).fit( z, d )
    return result

def yreg(z,y):
    result = standard_skl_model( LassoCV(cv = 10 , random_state = 0  ) ).fit( z, y )
    return result

DML2_lasso_cv = DML2_for_PLM(z, d, y, dreg, yreg, 10 , clu)

In [ ]:
# DML with cross-validated Lasso:
def dreg(z,d):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.5, max_iter = 100000 ) ).fit( z, d )
    return result

def yreg(z,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.5, max_iter = 100000 ) ).fit( z, y )
    return result

DML2_elnet = DML2_for_PLM(z, d, y, dreg, yreg, 10 , clu)

In [ ]:
#DML with cross-validated Lasso:
def dreg(z,d):
    result = standard_skl_model( ElasticNetCV( cv = 10 ,  random_state = 0 , l1_ratio = 0.0001 ) ).fit( z, d )
    return result

def yreg(z,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.0001 ) ).fit( z, y )
    return result

DML2_ridge = DML2_for_PLM(z, d, y, dreg, yreg, 10, clu)

Here we also compute DML with OLS used as the ML method

In [323]:
#DML with cross-validated Lasso:
def dreg(z,d):
    result = LinearRegression().fit( z, d )
    return result

def yreg(z,y):
    result = LinearRegression().fit( z, y )
    return result

DML2_ols = DML2_for_PLM(z, d, y, dreg, yreg, 10, clu)

0  
1  
Coefficient is 0.1738154996405108, SE is equal to 0.051287432641185


Next, we also apply Random Forest for comparison purposes.

### Random Forest


In [259]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [324]:
#DML with cross-validated Lasso:
def dreg(z,d):
    result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 65 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, d )
    return result

def yreg(z,y):
    result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 65 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, y )
    return result

DML2_RF = DML2_for_PLM(z, d, y, dreg, yreg, 10, clu)   # set to 2 due to computation time

<ipython-input-324-e21f895212cc>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 65 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, d )
<ipython-input-324-e21f895212cc>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 65 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, y )


0  


<ipython-input-324-e21f895212cc>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 65 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, d )
<ipython-input-324-e21f895212cc>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 65 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, y )


1  
Coefficient is 0.12647896945883857, SE is equal to 0.05663338110770135


We conclude that the gun ownership rates are related to gun homicide rates - if gun ownership increases by 1% relative
to a trend then the predicted gun homicide rate goes up by about 0.20% controlling for counties' characteristics.

Finally, let's see which method is actually better. We compute RMSE for predicting D and Y, and see which
of the methods works better.


In [325]:
mods = [DML2_ols, DML2_lasso, DML2_lasso_post , DML2_lasso_cv, DML2_ridge, DML2_elnet, DML2_RF]
mods_name = ["DML2_ols", "DML2_lasso", "DML2_lasso_post" , "DML2_lasso_cv", 'DML2_ridge', 'DML2_elnet', 'DML2_RF']

def mdl( model , model_name ):
    
    RMSEY = np.sqrt( np.mean( model[ 'ytil' ] ) ** 2 ) # I have some doubts about these equations...we have to recheck
    RMSED = np.sqrt( np.mean( model[ 'dtil' ] ) ** 2 ) # I have some doubts about these equations...we have to recheck
    
    result = pd.DataFrame( { model_name : [ RMSEY , RMSED ]} , index = [ 'RMSEY' , 'RMSED' ])
    return result

RES = [ mdl( model , name ) for model, name in zip( mods , mods_name ) ]
    

pr_Res = pd.concat( RES, axis = 1)

pr_Res

,DML2_ols,DML2_lasso,DML2_lasso_post,DML2_lasso_cv,DML2_ridge,DML2_elnet,DML2_RF
RMSEY,0.002580,0.002680,0.000099,2.277381e-19,7.401487e-19,2.846726e-19,0.000476
RMSED,0.001416,0.000652,0.000959,6.405133e-19,3.060230e-19,2.419717e-19,0.000320


#### This verfies that the function DML2_for_PLM has no errors

In [328]:
np.where(DML2_lasso_post[ 'ytil' ] == 0)[0].size

0

It looks like the best method for predicting D is Lasso, and the best method for predicting Y is CV Ridge.

In [ ]:
#DML with cross-validated Lasso:
def dreg(z,d):
    result = standard_skl_model( LassoCV(cv = 10 , random_state = 0 , alphas = [0]) ).fit( z, d )
    return result


def yreg(z,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 ,  random_state = 0 , l1_ratio = 0.0001 ) ).fit( z, y )
    return result

DML2_best = DML2_for_PLM(z, d, y , dreg, yreg, 10, clu)

In [331]:
table = np.zeros( ( 9 , 2 ))
table[ 0 , 0] = baseline_ols_table.iloc[ 1 , 0 ]
table[ 1 , 0] = control_ols_table.iloc[ 1 , 0 ]
table[ 2 , 0] = DML2_lasso['coef_est']
table[ 3 , 0] = DML2_lasso_post['coef_est']
table[ 4 , 0] = DML2_lasso_cv['coef_est']
table[ 5 , 0] = DML2_ridge['coef_est']
table[ 6 , 0] = DML2_elnet['coef_est']
table[ 7 , 0] = DML2_RF['coef_est']
table[ 8 , 0] = DML2_best['coef_est']
table[ 0 , 1] = baseline_ols_table.iloc[ 1 , 1 ]
table[ 1 , 1] = control_ols_table.iloc[ 1 , 1 ]
table[ 2 , 1] = DML2_lasso['se']
table[ 3 , 1] = DML2_lasso_post['se']
table[ 4 , 1] = DML2_lasso_cv['se']
table[ 5 , 1] = DML2_ridge['se']
table[ 6 , 1] = DML2_elnet['se']
table[ 7 , 1] = DML2_RF['se']
table[ 8 , 1] = DML2_best['se']

In [332]:
table = pd.DataFrame( table , index = [ "Baseline OLS", "Least Squares with controls", "Lasso", \
             "Post-Lasso", "CV Lasso","CV Elnet", "CV Ridge", "Random Forest", "Best" ] , \
            columns = ["Estimate","Standard Error"] )
table.round( 3 )

,Estimate,Standard Error
Baseline OLS,0.282,0.065
Least Squares with controls,0.192,0.053
Lasso,0.228,0.079
Post-Lasso,0.212,0.054
CV Lasso,0.231,0.057
CV Elnet,0.200,0.056
CV Ridge,0.207,0.056
Random Forest,0.126,0.057
Best,0.179,0.052


In [333]:
table.to_latex()

'\\begin{tabular}{lrr}\n\\toprule\n{} &  Estimate &  Standard Error \\\\\n\\midrule\nBaseline OLS                &  0.282304 &        0.064811 \\\\\nLeast Squares with controls &  0.191998 &        0.052921 \\\\\nLasso                       &  0.228288 &        0.078678 \\\\\nPost-Lasso                  &  0.212147 &        0.053999 \\\\\nCV Lasso                    &  0.230827 &        0.056527 \\\\\nCV Elnet                    &  0.200062 &        0.055897 \\\\\nCV Ridge                    &  0.207452 &        0.056077 \\\\\nRandom Forest               &  0.126479 &        0.056633 \\\\\nBest                        &  0.178552 &        0.051706 \\\\\n\\bottomrule\n\\end{tabular}\n'